Load all necessary imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

2024-03-26 13:32:49.586678: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 13:32:49.625123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 13:32:49.625153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 13:32:49.626216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 13:32:49.632925: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# import wandb
# import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="MyFirstComputerVision",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

# # simulate training
# epochs = 10
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset
    
#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

Data Loading

In [3]:
with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
    ls = list(hdf.keys())
    images = hdf.get('images')
    boxes = hdf.get('boxes')
    images = np.array(images)
    boxes = np.array(boxes)

Data Normalization

In [5]:
image_normalized = (images+1e-9)/9.26 # Normalize images and add noise


# Normalize bounding boxes
normalized_boxes = boxes / [1,64,64,64,64]

Model Train


In [6]:
from tensorflow.keras import layers, models



input_shape = (64, 64, 1)
num_classes = 280
num_coordinates = 4

images_np = image_normalized
#images_np = tf.expand_dims(images_np, axis=-1)
probabilities = np.array(normalized_boxes[:, :,:-4])
probabilities = tf.expand_dims(probabilities, axis=1)
boxes_np = np.array(normalized_boxes[:, :, 1:])
boxes_np = tf.expand_dims(boxes_np, axis=1)

dataset = tf.data.Dataset.from_tensor_slices((images_np,{'x_prob_reshape':probabilities,'x_boxes_reshape':boxes_np}))
dataset = dataset.shuffle(buffer_size=10000,reshuffle_each_iteration=True)
dataset = dataset.batch(128)

# # Assuming dataset has 10,000 examples and we want an 80/20 split
# total_items = 10000
# train_size = int(total_items * 0.6)
# val_size = total_items - train_size

# # Splitting the dataset
# train_dataset = dataset.take(train_size)
# val_dataset = dataset.skip(train_size)


# train_dataset = train_dataset.shuffle(buffer_size=train_size,reshuffle_each_iteration=True)
# train_dataset = train_dataset.batch(64)
# val_dataset = val_dataset.batch(64) 

# start a new wandb run to track this script



x_input = layers.Input(shape=input_shape)

x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) 

x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
 

x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)


x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
x = layers.MaxPool2D()(x)
x = layers.BatchNormalization()(x) # size: 8x8x


x = layers.Flatten()(x)
# Bounding box output
x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)

x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)


model = tf.keras.models.Model(x_input, [x_prob_reshape, x_boxes_reshape])


optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)  # Consider adjusting based on performance





model.compile(optimizer= optimizer, loss= {'x_prob_reshape': tf.keras.losses.BinaryCrossentropy(), 'x_boxes_reshape':tf.keras.losses.MeanSquaredError()}, metrics={'x_prob_reshape': 'accuracy'} )
num_epochs = 10


2024-03-26 13:33:17.969081: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21907 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [7]:
sweep_config = {
    'method': 'random'
    }

In [8]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [9]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [10]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

In [11]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [12]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [21]:
sweep_id = wandb.sweep(sweep_config, project="M23 Hyperparameter Tuning")

Create sweep with ID: 0bt130jx
Sweep URL: https://wandb.ai/alphanium/M23%20Hyperparameter%20Tuning/sweeps/0bt130jx


In [ ]:

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="MyFirstComputerVision",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 3e-5,
#     "architecture": "model",
#     "dataset": "dataset",
#     "epochs": "num_epochs",
#     }
# )
# # simulate training

# offset = random.random() / 5
# for epoch in range(2,num_epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset
    
#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

In [6]:
# model = tf.keras.models.load_model('M11overfittedmodel3variant.h5')


In [16]:



# model.fit(
#     dataset,
#     epochs=num_epochs 
   
# )





Epoch 1/10


2024-03-26 13:50:47.516518: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-26 13:50:47.602849: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 13:50:48.622823: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 13:50:49.512318: I external/local_xla/xla/service/service.cc:168] XLA service 0x7cecd44557e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 13:50:49.512356: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-26 13:50:49.525351: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711475449.672995   55592 device_compiler.

79/79 [==============================] - 21s 93ms/step - loss: 0.5040 - x_prob_reshape_loss: 0.3691 - x_boxes_reshape_loss: 0.1349 - x_prob_reshape_accuracy: 0.8537
Epoch 2/10
79/79 [==============================] - 6s 71ms/step - loss: 0.2388 - x_prob_reshape_loss: 0.1736 - x_boxes_reshape_loss: 0.0651 - x_prob_reshape_accuracy: 0.9482
Epoch 3/10
79/79 [==============================] - 6s 72ms/step - loss: 0.1882 - x_prob_reshape_loss: 0.1345 - x_boxes_reshape_loss: 0.0537 - x_prob_reshape_accuracy: 0.9566
Epoch 4/10
79/79 [==============================] - 6s 71ms/step - loss: 0.1692 - x_prob_reshape_loss: 0.1191 - x_boxes_reshape_loss: 0.0501 - x_prob_reshape_accuracy: 0.9581
Epoch 5/10
79/79 [==============================] - 6s 70ms/step - loss: 0.1578 - x_prob_reshape_loss: 0.1096 - x_boxes_reshape_loss: 0.0483 - x_prob_reshape_accuracy: 0.9592
Epoch 6/10
79/79 [==============================] - 6s 71ms/step - loss: 0.1443 - x_prob_reshape_loss: 0.0975 - x_boxes_reshape_loss: 0

In [23]:
def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        config = run.config
        # Build, train, and evaluate your model
        model = model(config)
        model.fit(dataset, epochs=num_epochs)
        # Optionally log model performance and save the model

wandb.agent(sweep_id, train, count=12)  # Run the sweep agent


wandb: Agent Starting Run: omkow137 with config:
wandb: 	batch_size: 96
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.06272054278490398
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Traceback (most recent call last):
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment


Run omkow137 errored:
Traceback (most recent call last):
  File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment

wandb: ERROR Run omkow137 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
wandb: ERROR     model = model(config)
wandb: ERROR UnboundLocalError: local variable 'model' referenced before assignment
wandb: ERROR 
wandb: Agent Starting Run: au1nfahk with config:
wandb: 	batch_size: 144
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.

Traceback (most recent call last):
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment


Run au1nfahk errored:
Traceback (most recent call last):
  File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment

wandb: ERROR Run au1nfahk errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
wandb: ERROR     model = model(config)
wandb: ERROR UnboundLocalError: local variable 'model' referenced before assignment
wandb: ERROR 
wandb: Agent Starting Run: wbaw45hk with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.0

Traceback (most recent call last):
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment


Run wbaw45hk errored:
Traceback (most recent call last):
  File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
    model = model(config)
UnboundLocalError: local variable 'model' referenced before assignment

wandb: ERROR Run wbaw45hk errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_55414/2795004664.py", line 6, in train
wandb: ERROR     model = model(config)
wandb: ERROR UnboundLocalError: local variable 'model' referenced before assignment
wandb: ERROR 
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this ch

In [30]:
#model.save('M11overfittedmodel3variant.h5')



In [31]:
#model.save('M11overfittedmodel3variant.keras')

In [10]:
boxes.shape

(10000, 280, 5)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((images_np,{'x_prob_reshape':probabilities,'x_boxes_reshape':boxes_np}))

# # Assuming dataset has 10,000 examples and we want an 80/20 split
# total_items = 10000
# train_size = int(total_items * 0.6)
# # val_size = total_items - train_size

# # Splitting the dataset
# train_dataset = dataset.take(train_size)
# # val_dataset = dataset.skip(train_size)


# # train_dataset = train_dataset.shuffle(buffer_size=train_size,reshuffle_each_iteration=True)
# train_dataset = train_dataset.batch(6000)
# inputs,targets = next(iter(train_dataset))
dataset = dataset.batch(10000)
inputs,targets = next(iter(dataset))
output = model.predict(inputs)

313/313 [==============================] - 3s 8ms/step


In [11]:
targets['x_boxes_reshape'][0]

<tf.Tensor: shape=(1, 280, 4), dtype=float64, numpy=
array([[[0.734375, 0.015625, 0.734375, 0.03125 ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        ...,
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ]]])>

In [12]:
output[1][0]

array([[[7.24306941e-01, 3.01006213e-02, 7.41804957e-01, 3.33228447e-02],
        [2.24880949e-02, 7.51386629e-03, 2.32093632e-02, 7.64035154e-03],
        [4.27702442e-03, 6.26132824e-03, 4.65806713e-03, 6.90292520e-03],
        ...,
        [2.88477138e-04, 1.39197495e-04, 1.47837753e-04, 1.58194220e-04],
        [1.42245786e-04, 2.08800062e-04, 1.84095348e-04, 1.41638870e-04],
        [1.16996292e-04, 1.13774906e-04, 1.76820322e-04, 1.51523607e-04]]],
      dtype=float32)

In [9]:
import tensorflow as tf

r = np.random.randint(0,100)
tensor1 = tf.constant(targets['x_prob_reshape'], dtype=tf.float64)

tensor2 = tf.constant(output[0], dtype=tf.float32)


tensor2 = tf.cast(tensor2, tf.float64)


mse_loss_fn =  tf.keras.losses.BinaryCrossentropy()
mse_loss = mse_loss_fn(tensor1, tensor2)

print("MSE Loss:", mse_loss.numpy())


MSE Loss: 1.3558886642682864e-07


In [2]:
def visualize_bounding_boxes(image, probability_vector, bounding_box_coordinates, threshold=0.9):
    """
    Visualizes bounding boxes on an image based on a probability vector.

    Parameters:
    - image: A 3D tensor representing the image.
    - probability_vector: A 1D tensor representing the probabilities associated with bounding boxes.
    - bounding_box_coordinates: A 2D tensor representing bounding box coordinates.
    - threshold: Probability threshold for visualization.

    Returns:
    None (displays the image with bounding boxes).
    """
    # Convert to NumPy arrays for easier handling
    image_np = image
    prob_vector_np = probability_vector
    bbox_coordinates_np = bounding_box_coordinates
   # Denormalize image if necessary (adjust based on your normalization method)
    denormalized_image = image_np  # Modify if normalization was applied during training

    # Visualize the image
    plt.figure(figsize=(4, 4))
    plt.imshow(denormalized_image, cmap='gray')
    plt.title("Bounding Box Visualization")

    # Plot bounding boxes based on probability threshold
    for i in range(len(prob_vector_np)):
        prob = prob_vector_np[i]
        bbox = bbox_coordinates_np[i]
        if prob > threshold:
            # Denormalize bounding box coordinates if necessary
            denormalized_bbox = bbox  # Modify if normalization was applied during training
            y1, x1, y2, x2 = denormalized_bbox
            plt.plot([x1, x2, x2, x1, x1],[y1, y1, y2, y2, y1],
                     
                     color='r', linewidth=2, label='Bounding Box')

   






   
    plt.show()

t = np.random.randint(0,100)


visualize_bounding_boxes(tf.convert_to_tensor(inputs[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(output[1][t,0,:,:])*[64,64,64,64]) ##myprediction
visualize_bounding_boxes(tf.convert_to_tensor(image_normalized[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(boxes_np[t,0,:,:])*[64,64,64,64]) ##myprediction





NameError: name 'np' is not defined